## CONFORMAL MAPPING


In [7]:
#suppongo di avere una lista di gap e una lista di cpw la len(gap)=len(cpw)+1
import numpy as np
from scipy import integrate
from scipy.optimize import root


def branch_point_coordinates(gap,cpw):
    a=[] #branch points destri
    b=[] #branch points sinistri
    a.append(0) #il punto a_0 ha sempre coordinata 0 che è dove pongo la mia orgine
    b.append(gap[0])
    x=0
    y=gap[0]
    for i in range(len(cpw)):
        x+=(gap[i]+cpw[i])
        y+=(gap[i+1]+cpw[i])
        a.append(x) #coordinate dei punti a che sono alla destra dei conduttori
        b.append(y) #coordinate dei punti b che sono alla sinistra dei conduttori

    a_coordinates=[complex(x) for x in a]
    b_coordinates=[complex(x) for x in b]
    return a_coordinates,b_coordinates

def conformal_mapping(a_coordinates,b_coordinates,c_coordinates):
    def w(z):
        num=1
        for c in c_coordinates:
            num=num*(z-c)
        den1=1
        for a in a_coordinates:
            den1=den1*(z-a)**0.5
        den2=1
        for b in b_coordinates:
            den2=den2*(z-b)**0.5
        den=den1*den2
        func=num/den
        return func
    def integrate_complex(func, a, b):
        real_part, _ = integrate.quad(lambda t: np.real(func(t)), a+1e-9, b-1e-9)
        imag_part, _ = integrate.quad(lambda t: np.imag(func(t)), a+1e-9, b-1e-9)
        return real_part + 1j * imag_part
    
    ap=[] #nuove coordinate di a dopo il conformal mapping
    bp=[] #nuove coordinate di b dopo il conformal mapping
    ap.append(complex(0)) #il primo resta zero perché è l'origine
    val = integrate_complex(w, a_coordinates[0].real, b_coordinates[0].real)
    bp.append(val)
    for i in range(1,len(b_coordinates)):
        val += integrate_complex(w, b_coordinates[i-1].real, a_coordinates[i].real)
        ap.append(val)
        val += integrate_complex(w, a_coordinates[i].real, b_coordinates[i].real)
        bp.append(val)
    return ap,bp

def find_c(a_coordinates,b_coordinates, metal_i):
    def c_solve(c_coordinates): #c_coordinates are real number
        def w(z):
            num=1
            for c in c_coordinates:
                num=num*(z-complex(c))
            den1=1
            for a in a_coordinates:
                den1=den1*(z-a)**0.5
            den2=1
            for b in b_coordinates:
                den2=den2*(z-b)**0.5
            den=den1*den2
            func=num/den
            return func
        def integral_on_gap(func,a,b):
            imag_part, _ = integrate.quad(lambda t: np.imag(func(t)), a+1e-9, b-1e-9)
            return imag_part
        constraints=[]
        for j in range(len(a_coordinates)):
            if j!=metal_i and j!=(metal_i+1):
                f=integral_on_gap(w,a_coordinates[j].real,b_coordinates[j].real)
                constraints.append(f)
        return constraints
    
    c_coordinates_guest=[]
    for j in range(len(a_coordinates)):
        if j!=metal_i and j!=(metal_i+1):
            c=(b_coordinates[j].real+a_coordinates[j].real)/2
            c_coordinates_guest.append(c)
    sol=root(c_solve,c_coordinates_guest)
    if not sol.success:
        raise RuntimeError("Root finding failed: " + sol.message)
    c_coordinates=[complex(x) for x in sol.x]
    return c_coordinates



    

    
    


In [10]:
gap=[10,10,5,5]
cpw=[15,5,15]
a,b=branch_point_coordinates(gap,cpw)
c=find_c(a,b,0)

# ap,bp=conformal_mapping(a,b,c)

C:\Users\simyz\AppData\Local\Temp\ipykernel_23644\3124834904.py:71: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  imag_part, _ = integrate.quad(lambda t: np.imag(func(t)), a+1e-9, b-1e-9)


In [1]:
import numpy as np
from scipy import integrate
from scipy.optimize import root
from scipy.constants import epsilon_0, mu_0


def branch_point_coordinates(gap,cpw):
    a=[] #branch points destri
    b=[] #branch points sinistri
    a.append(0) #il punto a_0 ha sempre coordinata 0 che è dove pongo la mia orgine
    b.append(gap[0])
    x=0
    y=gap[0]
    for i in range(len(cpw)):
        x+=(gap[i]+cpw[i])
        y+=(gap[i+1]+cpw[i])
        a.append(x) #coordinate dei punti a che sono alla destra dei conduttori
        b.append(y) #coordinate dei punti b che sono alla sinistra dei conduttori

    a_coordinates=[complex(x) for x in a]
    b_coordinates=[complex(x) for x in b]
    return a_coordinates,b_coordinates

def conformal_mapping(a_coordinates,b_coordinates,c_coordinates):
    def integral_by_part(z0,z1):
        def f1(z):
            u=1
            for c in c_coordinates:
                u=u*(z-c)
            for a in a_coordinates:
                if a!=z0 and a!=z1:
                    u=u*(z-a)**(-0.5)
            for b in b_coordinates:
                if b!=z0 and b!=z1:
                    u=u*(z-b)**(-0.5)
            v=np.log(z-((z0+z1)/2)+(z-z0)**0.5*(z-z1)**0.5)
            f1=u*v
            return f1
        def f2(z):
            v=np.log(z-((z0+z1)/2)+(z-z0)**0.5*(z-z1)**0.5)
            prod1 = np.prod([z - c for c in c_coordinates])
            prod2 = np.prod([(z - a)**(-0.5) for a in a_coordinates if a != z0 and a != z1]) \
                * np.prod([(z - b)**(-0.5) for b in b_coordinates if b != z0 and b != z1])
            u_prime=0
            for c in c_coordinates:
                prod3 = np.prod([z - d for d in c_coordinates if d != c])
                u_prime += prod3 * prod2
            sum_term = 0
            for a in a_coordinates:
                if a != z0 and a != z1:
                    sum_term += (z - a)**(-1)
            for b in b_coordinates:
                if b != z0 and b != z1:
                    sum_term += (z - b)**(-1)

            u_prime -= (prod1 * prod2 * sum_term) / 2
            f2=v*u_prime
            return f2
        integral_part = f1(z1)-f1(z0)
        def z(t):
            return t * (z1 - z0) + z0  # Parametrizzazione del segmento

        real_part, _ = integrate.quad(lambda t: f2(z(t)).real, 0, 1)
        imag_part, _ = integrate.quad(lambda t: f2(z(t)).imag, 0, 1)

        numerical_part = real_part + 1j * imag_part
        return integral_part - (z1-z0)*numerical_part   
                    
    ap=[] #nuove coordinate di a dopo il conformal mapping
    bp=[] #nuove coordinate di b dopo il conformal mapping
    ap.append(complex(0)) #il primo resta zero perché è l'origine
    val = integral_by_part(a_coordinates[0], b_coordinates[0])
    bp.append(val)
    for i in range(1,len(b_coordinates)):
        val +=integral_by_part( b_coordinates[i-1], a_coordinates[i])
        ap.append(val)
        val += integral_by_part( a_coordinates[i], b_coordinates[i])
        bp.append(val)
    return ap,bp


def find_c(a_coordinates,b_coordinates, metal_i):
    def c_solve(c_coordinates): #c_coordinates are real number
        def integral_by_part(z0,z1):
            def f1(z):
                u=1
                for c in c_coordinates:
                    u=u*(z-complex(c))
                for a in a_coordinates:
                    if a!=z0 and a!=z1:
                        u=u*(z-a)**(-0.5)
                for b in b_coordinates:
                    if b!=z0 and b!=z1:
                        u=u*(z-b)**(-0.5)
                v=np.log(z-((z0+z1)/2)+(z-z0)**0.5*(z-z1)**0.5)
                f1=u*v
                return f1
            
            def f2(z):
                v=np.log(z-((z0+z1)/2)+(z-z0)**0.5*(z-z1)**0.5)
                prod1 = np.prod([z - complex(c) for c in c_coordinates])
                prod2 = np.prod([(z - a)**(-0.5) for a in a_coordinates if a != z0 and a != z1]) \
                    * np.prod([(z - b)**(-0.5) for b in b_coordinates if b != z0 and b != z1])
                u_prime=0
                for c in c_coordinates:
                    prod3 = np.prod([z - complex(d) for d in c_coordinates if d != c])
                    u_prime += prod3 * prod2
                sum_term = 0
                for a in a_coordinates:
                    if a != z0 and a != z1:
                        sum_term += (z - a)**(-1)
                for b in b_coordinates:
                    if b != z0 and b != z1:
                        sum_term += (z - b)**(-1)

                u_prime -= (prod1 * prod2 * sum_term) / 2
                f2=v*u_prime
                return f2
            
            integral_part = f1(z1)-f1(z0)
            def z(t):
                return t * (z1 - z0) + z0  # Parametrizzazione del segmento

            real_part, _ = integrate.quad(lambda t: f2(z(t)).real, 0, 1)
            imag_part, _ = integrate.quad(lambda t: f2(z(t)).imag, 0, 1)

            numerical_part = real_part + 1j * imag_part
            return integral_part - (z1-z0)*numerical_part
          
        constraints=[]
        for j in range(len(a_coordinates)):
            if j!=metal_i and j!=(metal_i+1):
                f=integral_by_part(a_coordinates[j],b_coordinates[j])
                constraints.append(f.imag)
        return constraints
    
    c_coordinates_guest=[]
    for j in range(len(a_coordinates)):
        if j!=metal_i and j!=(metal_i+1):
            c=(b_coordinates[j].real+a_coordinates[j].real)/2
            c_coordinates_guest.append(c)
    sol=root(c_solve,c_coordinates_guest)
    if not sol.success:
        raise RuntimeError("Root finding failed: " + sol.message)
    c_coordinates=[complex(x) for x in sol.x]
    return c_coordinates

In [2]:
gap=[10,10,5,5]
cpw=[15,5,15]
a,b=branch_point_coordinates(gap,cpw)
c=find_c(a,b,0)

ap,bp=conformal_mapping(a,b,c)

In [4]:
print(ap)
print(bp)

[0j, np.complex128(-0.18639045546096733+0.1298831674509596j), np.complex128(-0.1387346076964095-3.209238430557093e-16j), np.complex128(-0.11683059429559733-1.1796119636642288e-16j)]
[np.complex128(-4.85722573273506e-17+0.12988316745095976j), np.complex128(-0.18639045546096733-3.3306690738754696e-16j), np.complex128(-0.1387346076964095-1.196959198423997e-16j), np.complex128(-0.11683059429559733-1.6219664500383146e-16j)]


## CAPACITANCE AND INDUCTANCE MATRICIES

In [3]:
from scipy.constants import epsilon_0, mu_0
epsilon_r=11.7

def CL_matrices(gap,cpw):
    a,b=branch_point_coordinates(gap,cpw)
    C=np.zeros((len(cpw),len(cpw)))
    for j in range(len(cpw)):
        c=find_c(a,b,j)
        ap,bp=conformal_mapping(a,b,c)
        for i in range(len(cpw)):
            C[i,j]=(epsilon_r+1)*epsilon_0*(bp[i].real-ap[i+1].real)/bp[j].imag

    eps_eff=(epsilon_r+1)/2
    c=3e8
    c_l=c/np.sqrt(eps_eff)
    L=np.linalg.inv(C)/c_l**2

    return C,L



In [18]:
gap=[10,10,10,10]
cpw=[15,5,15]
C,L=CL_matrices(gap,cpw)

print(C,'\n')
print(L,'\n')
print(C@L,'\n')
print(np.linalg.inv(C)@L,'\n')

C = np.delete(C, 1, axis=0)  # togli riga r
C = np.delete(C, 1, axis=1)  # togli colonna c
L = np.delete(L, 1, axis=0)  # togli riga r
L = np.delete(L, 1, axis=1)  # togli colonna c

print(c_l*L)

[[ 1.60767224e-10 -4.40290987e-11 -1.75414133e-11]
 [-4.40290987e-11  1.18153890e-10 -4.40290987e-11]
 [-1.75414133e-11 -4.40290987e-11  1.60767224e-10]] 

[[5.17358144e-07 2.38126741e-07 1.21664718e-07]
 [2.38126741e-07 7.74621703e-07 2.38126741e-07]
 [1.21664718e-07 2.38126741e-07 5.17358144e-07]] 

[[ 7.05555556e-17 -2.05145187e-33 -1.29514118e-33]
 [-6.11995908e-34  7.05555556e-17 -6.30238007e-33]
 [-1.20785074e-33  1.68619219e-34  7.05555556e-17]] 

[[ 4807.07855108  4771.08528796  2587.9295875 ]
 [ 4771.08528796 10111.85393048  4771.08528796]
 [ 2587.9295875   4771.08528796  4807.07855108]] 

[[61.5921951  14.48435112]
 [14.48435112 61.5921951 ]]


## ADMITTANCE MATRIX

In [19]:
from scipy.linalg import sqrtm
epsilon_r=11.7
eps_eff=(epsilon_r+1)/2
c=3e8
c_l=c/np.sqrt(eps_eff)
l=0.2e-3
v=c_l
gap=[10,10,10,10]
cpw=[15,5,15]

def Z(gap,cpw):
    C,L=CL_matrices(gap,cpw)
    Z=sqrtm(L @ np.linalg.inv(C))
    Z= np.delete(Z, 1, axis=0)  # togli riga r
    Z = np.delete(Z, 1, axis=1)  # togli colonna c
    #Z=v*L
    print(Z,'\n')
    return Z
Z_matrix=Z(gap,cpw)
def Y(z):
    Z_matrix_inv=np.linalg.inv(Z_matrix)
    print(Z_matrix_inv,'\n')
    exp_z=np.exp(l*z/v)
    exp_mz=np.exp(-l*z/v)
    imag_part=exp_z-exp_mz
    Y=np.zeros((4,4), dtype=np.complex128)
    for i in range(4):
        bound=np.zeros(4, dtype=np.complex128)
        V_plus=np.zeros(2,dtype=np.complex128)
        V_minus=np.zeros(2,dtype=np.complex128)
        bound[i]=1
        V_plus[0]=(bound[0]*exp_z-bound[1])/imag_part
        V_plus[1]=(bound[2]*exp_z-bound[3])/imag_part
        V_minus[0]=(-bound[0]*exp_mz+bound[1])/imag_part
        V_minus[1]=(-bound[2]*exp_mz+bound[3])/imag_part
        print(V_plus, V_minus,'\n')

        I_plus=Z_matrix_inv @ V_plus
        I_minus=-Z_matrix_inv @ V_minus
        print(I_plus, I_minus,'\n')
        Y[0,i]=I_plus[0]+I_minus[0]
        Y[1,i]=-(I_plus[0]*exp_mz+I_minus[0]*exp_z)
        Y[2,i]=I_plus[1]+I_minus[1]
        Y[3,i]=-(I_plus[1]*exp_mz+I_minus[1]*exp_z)
    return Y

admittance=Y(1j*2*np.pi*4*1e9)
print(admittance)

[[61.5921951  14.48435112]
 [14.48435112 61.5921951 ]] 

[[ 0.01718627 -0.00404162]
 [-0.00404162  0.01718627]] 

[0.5-11.83522083j 0.  +0.j        ] [0.5+11.83522083j 0.  +0.j        ] 

[ 0.00859314-0.20340332j -0.00202081+0.04783342j] [-0.00859314-0.20340332j  0.00202081+0.04783342j] 

[0.+11.84577782j 0. +0.j        ] [0.-11.84577782j 0. +0.j        ] 

[0.+0.20358475j 0.-0.04787608j] [0.+0.20358475j 0.-0.04787608j] 

[0.  +0.j         0.5-11.83522083j] [0.  +0.j         0.5+11.83522083j] 

[-0.00202081+0.04783342j  0.00859314-0.20340332j] [ 0.00202081+0.04783342j -0.00859314-0.20340332j] 

[0. +0.j         0.+11.84577782j] [0. +0.j         0.-11.84577782j] 

[0.-0.04787608j 0.+0.20358475j] [0.-0.04787608j 0.+0.20358475j] 

[[ 0.-0.40680664j  0.+0.40716951j  0.+0.09566683j  0.-0.09575217j]
 [-0.+0.40716951j -0.-0.40680664j -0.-0.09575217j -0.+0.09566683j]
 [ 0.+0.09566683j  0.-0.09575217j  0.-0.40680664j  0.+0.40716951j]
 [-0.-0.09575217j -0.+0.09566683j -0.+0.40716951j -0.-0.40680

In [24]:
def Z(gap,cpw):
    C,L=CL_matrices(gap,cpw)
    Z=sqrtm(L @ np.linalg.inv(C))
    if Z.shape[0]==3:
        Z= np.delete(Z, 1, axis=0)  # togli riga r
        Z = np.delete(Z, 1, axis=1)  # togli colonna c
    return Z

gap=[10,10,10,10]
cpw=[15,5,15]

Z_matrix=Z(gap,cpw)
def Y(z):
    Z_matrix_inv=np.linalg.inv(Z_matrix)
    exp_z=np.exp(l*z/v)
    exp_mz=np.exp(-l*z/v)
    imag_part=exp_z-exp_mz
    dim=Z_matrix.shape[0]
    Y=np.zeros((2*dim,2*dim), dtype=np.complex128)
    for i in range(2*dim):
        bound=np.zeros(2*dim, dtype=np.complex128)
        V_plus=np.zeros(dim,dtype=np.complex128)
        V_minus=np.zeros(dim,dtype=np.complex128)
        #if dim!=3 or (i!=2 and i!=3):
        bound[i]=1
        for j in range(dim):
            V_plus[j]=(bound[2*j]*exp_z-bound[2*j+1])/imag_part
            V_minus[j]=(-bound[2*j]*exp_mz+bound[2*j+1])/imag_part
       
        I_plus=Z_matrix_inv @ V_plus
        I_minus=-Z_matrix_inv @ V_minus
        for j in range(dim):
            Y[2*j,i]=I_plus[j]+I_minus[j]
            Y[2*j+1,i]=-(I_plus[j]*exp_mz+I_minus[j]*exp_z)
     
    return Y

admittance=Y(1j*2*np.pi*4*1e9)
print(admittance)

[[ 0.-0.40680664j  0.+0.40716951j  0.+0.09566683j  0.-0.09575217j]
 [-0.+0.40716951j -0.-0.40680664j -0.-0.09575217j -0.+0.09566683j]
 [ 0.+0.09566683j  0.-0.09575217j  0.-0.40680664j  0.+0.40716951j]
 [-0.-0.09575217j -0.+0.09566683j -0.+0.40716951j -0.-0.40680664j]]


In [25]:
print(Z(gap,cpw))

[[61.5921951  14.48435112]
 [14.48435112 61.5921951 ]]


In [26]:
C,L=CL_matrices(gap,cpw)

In [28]:
print(c_l*L)

[[61.5921951  28.3493144  14.48435112]
 [28.3493144  92.21977378 28.3493144 ]
 [14.48435112 28.3493144  61.5921951 ]]
